# Optional - Advanced Solution 🧙🧙

The last solution is perfectly valid and we applied the rules from the lectures strictly. Now that we are more comfortable with Preprocessing with python, let's take a step back and see what we could have done differently by digging into the interpretation of the variables a little deeper.

1. Load the titanic dataset again

In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [6]:
path = "titanic.csv"
df = pd.read_csv(path)

In [9]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [126]:
pip install pandas.dq

Note: you may need to restart the kernel to use updated packages.


In [133]:
from pandas_dq import dq_report

In [139]:
dq_report(df)

This is a summary report. Change verbose to 1 to see more details on each DQ issue.
    All variables classified into correct types.


C:\Users\dsgat\AppData\Local\Temp\ipykernel_17700\3031772947.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dq_report(df)


,DQ Issue
The Good News,"There are no duplicate rows in this dataset, There are no duplicate columns in this datatset, There are no zero-variance or low information columns in the dataset., There are no date-time vars in this dataset, There are no columns with infinite values in this dataset , There are no highly correlated columns in the dataset., There is no target given. Hence no target leakage columns detected in the dataset"
The Bad News,"There are ID columns in the dataset. Remove them before modeling using Fix_DQ., There are 2 columns with mixed data types in the dataset. Convert them to a single type or split them into multiple columns., There are 5 numerical columns, some with outliers. Remove them or use robust statistics., There are 2 columns with high cardinality (>30 categories) in the dataset. Reduce them using encoding techniques or feature selection methods."


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
PassengerId,int64,0.000000,100,1.0,891.0,Possible ID column: drop before modeling step.
Survived,int64,0.000000,0,0.0,1.0,No issue
Pclass,int64,0.000000,0,1.0,3.0,No issue
Name,object,0.000000,100,,,No issue
Sex,object,0.000000,0,,,No issue
Age,float64,19.865320,NA,0.42,80.0,"177 missing values. Impute them with mean, median, mode, or a constant value such as 123., Column has 11 outliers greater than upper bound (64.81) or lower than lower bound(-6.69). Cap them or remove them."
SibSp,int64,0.000000,0,0.0,8.0,Column has 46 outliers greater than upper bound (2.50) or lower than lower bound(-1.50). Cap them or remove them.
Parch,int64,0.000000,0,0.0,6.0,Column has 213 outliers greater than upper bound (0.00) or lower than lower bound(0.00). Cap them or remove them.
Ticket,object,0.000000,76,,,Possible high cardinality column with 681 unique values: Use hash encoding or text embedding to reduce dimension.
Fare,float64,0.000000,NA,0.0,512.3292,Column has 116 outliers greater than upper bound (65.63) or lower than lower bound(-26.72). Cap them or remove them.


Let's explore the features more in details and try to extract more information than previously:

**A. Preprocessing to be planned with pandas**

**Unnecessary columns for prediction, to be thrown away** :
- _PassengerId_ and _Name_ are passenger identifiers, we won't use them for prediction (these columns don't contain any information)

<Note type="tip" title="Actually, _Name_ contains useful information !">

As it is true that _Name_ cannot be used as such for prediction, it contains valuable information on the socio-economic background of the passenger in the form of their title. We will try and extract a _Title_ variable from the variable _Name_

</Note>

- _Ticket_ and _Cabin_ have too many different modalities, they might not be very useful and if we had to pass them in OneHotEncoding, they would make the number of columns explode in relation to the number of rows.

<Note type="tip" title="We can do something with the _Cabin_ variable !">

_Ticket_ and _Cabin_ do have way too many modalities in order to be useful for prediction, however, the _Cabin_ variable can easily be used after a slight transformation : let's create a new variable _HasCabin_ which is equal to 1 when the passenger has a cabin number and 0 otherwise.

</Note>

**Columns with too many missing values, to be discarded** : Cabin


**Target variable/target (Y) that we will try to predict, to separate from the others** : Survived

**------------**

**B. Preprocessings to be planned with scikit-learn**.

**Explanatory variables (X)**
We need to identify which columns contain categorical variables and which columns contain numerical variables, as they will be treated differently.

- Categorical variables : Sex, Embarked, HasCabin, Title
- Numerical variables : Class, Age, Bbsp, Parch, Fare.

In this dataset, we have both types of variables. It will thus be necessary to plan to create a numeric_transformer (which will call the StandardScaler class) and a categorical_transformer (which will call the OneHotEncoder class). Moreover, as we observe missing values in the _Age_ and _Embarked_ columns, we will have to plan to call the SimpleImputer class to handle the missing values. 

**Target variable Y**
Here, the target variable Y is categorical (survival vs. death) but we notice that it is already encoded in numbers (1 vs. 0). It will therefore not be necessary to go through a label encoding step.

## Preprocessing - pandas part ##
2. Create a column _HasCabin_ in the dataset that is constant equal to 1

In [140]:
df['HasCabin'] = 1

In [141]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,HasCabin,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,1,Rare
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,1,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,Mr


3. Using a mask, change the value of the variable _HasCabin_ to 0 wherever Cabin is missing.

In [142]:
import pandas as pd
import numpy as np
df['HasCabin'] = df['Cabin'].apply(lambda x: 0 if pd.isnull(x) else 1)

In [143]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,HasCabin,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,Mr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0,Rare
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,Miss
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0,Miss
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1,Mr


In [144]:
from pandas_dq import dq    # import the package
pip install pandas-dq


SyntaxError: invalid syntax (2620378449.py, line 2)

5. Create a column _Title_ that only contains the title extracted from the _Name_ variable. 

<Note type="tip" title="Remember pandas handles columns of strings efficiently">

Some method from [the str module](https://docs.python.org/3.3/library/stdtypes.html?highlight=split) can be helpful 😉
You can create a function that allows to extract the title from one element of the column, and then use the `apply()` method to apply this function to the whole column.

</Note>

In [ ]:
def get_title(name):
    return name.split(', ')[1].split('.')[0]

In [ ]:
df['Title'] = df['Name'].apply(get_title)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,HasCabin,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,Mr


6. Display all the possible values and number of instances of each of these values in your dataset for the new _Title_ variable.

In [ ]:
df["Title"].value_counts()

Title
Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: count, dtype: int64

7. Some of these values represent only very few instances, and other values seem to represent the similar categories of people. Bring the similar categories under one name, and create a new category called _Rare_ that will represent all the underrepresented modalities.

In [ ]:
# Group all synonimous titles under one category
df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

# Group all non-common titles into one single grouping "Rare"
df['Title'] = df['Title'].replace(['Lady', 'the Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

df['Title'].value_counts()

Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64

8. Now that we are done squeezing some extra information out of our variables, let's reproduce all the subsequent steps from the first solution and let's compare our models' performances.

In [ ]:
# Drop useless columns / columns with too many missing values
useless_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin']

print("Dropping useless columns...")
dataset = df.drop(useless_cols, axis=1)
print("...Done.")
dataset.head()

Dropping useless columns...
...Done.


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,HasCabin,Title
0,0,3,male,22.0,1,0,7.2500,S,0,Mr
1,1,1,female,38.0,1,0,71.2833,C,1,Mrs
2,1,3,female,26.0,0,0,7.9250,S,0,Miss
3,1,1,female,35.0,1,0,53.1000,S,1,Mrs
4,0,3,male,35.0,0,0,8.0500,S,0,Mr


In [ ]:
# Separate target variable Y from features X
target_name = 'Survived'

print("Separating labels from features...")
Y = df.loc[:,target_name]
X = df.drop(target_name, axis = 1) # Keeping all columns
print("...Done.")
print(Y.head())
print()
print(X.head())

Separating labels from features...
...Done.
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

   PassengerId  Pclass                                               Name  \
0            1       3                            Braund, Mr. Owen Harris   
1            2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2            3       3                             Heikkinen, Miss. Laina   
3            4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4            5       3                           Allen, Mr. William Henry   

      Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked  \
0    male  22.0      1      0         A/5 21171   7.2500   NaN        S   
1  female  38.0      1      0          PC 17599  71.2833   C85        C   
2  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S   
3  female  35.0      1      0            113803  53.1000  C123        S   
4    male  35.0      0      0            373450   8.0

In [ ]:
from sklearn.model_selection import train_test_split
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create pipeline for numeric features
numeric_features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] # Names of numeric columns in X_train/X_test
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # missing values in Age will be replaced by columns' mean
    ('scaler', StandardScaler())
])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Create pipeline for categorical features
categorical_features = ['Sex', 'Embarked', 'HasCabin', 'Title'] # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
# Use ColumnTranformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5,:])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) # Don't fit again !!
print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on train set...
     PassengerId  Pclass                                            Name  \
545          546       1                    Nicholson, Mr. Arthur Ernest   
37            38       3                        Cann, Mr. Ernest Charles   
214          215       3                             Kiernan, Mr. Philip   
40            41       3  Ahlin, Mrs. Johan (Johanna Persdotter Larsson)   
236          237       2                               Hold, Mr. Stephen   

        Sex   Age  SibSp  Parch      Ticket    Fare Cabin Embarked  HasCabin  \
545    male  64.0      0      0         693  26.000   NaN        S         0   
37     male  21.0      0      0  A./5. 2152   8.050   NaN        S         0   
214    male   NaN      1      0      367229   7.750   NaN        Q         0   
40   female  40.0      1      0        7546   9.475   NaN        S         0   
236    male  44.0      1      0       26707  26.000   NaN        S         0   

    Title  
545    M

### Training model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Train model
model = LogisticRegression()

print("Training model...")
model.fit(X_train, Y_train) # Training is always done on train set !!
print("...Done.")

Training model...
...Done.


### Predictions

In [ ]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = model.predict(X_train)
print("...Done.")
print(Y_train_pred[0:5])
print()

Predictions on training set...
...Done.
[0 0 0 1 0]



In [ ]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = model.predict(X_test)
print("...Done.")
print(Y_test_pred[0:5])
print()

Predictions on test set...
...Done.
[0 0 0 1 1]



### Performances evaluation

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# Print scores
print("Accuracy on training set : ", accuracy_score(Y_train, Y_train_pred))
print("Accuracy on test set : ", accuracy_score(Y_test, Y_test_pred))

Accuracy on training set :  0.8282694848084544
Accuracy on test set :  0.8208955223880597


Tada 🥳 If you worked well, the score has improved a bit!
This example shows that by adding a little additional information to a model, it is possible to create a significant impact on the performances of the predictive model. Knowing and applying the preprocessing guidelines is great but always remember to check for two important things before you proceed :

* If a variable contains missing values, ask yourself why this value is missing and whether you could use it as information to feed the model with. In the above example, the fact that a passenger does not have a cabin number simply means that they have no cabin. It is very common for missing values to contain hidden meaning, completely random missing values (caused by a bug or other unpredictable causes) are very rare.

* When a non-numerical variable is not usable as is, always ask yourself whether you could still extract some information from it. Here the _Name_ variable cannot be used, however it mentions the passenger's title which can be useful information.